In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
from data import fut_list, fut_read, stock_read
from util import adf_test, data_generator, significant, johansen_cointegration_test, EG

# Config

In [2]:
fut = 'CU9999.XSGE'
feature = 'ChangeRatio'

# DATA
* 数据基础已经全部存在data里了。只需进一步处理得到想要的时间序列。

In [3]:
fut_df = fut_read(fut)
stock_df = stock_read(fut)

/Users/hejizhang/Downloads/主连日线-2018年后/data.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fut['ChangeRatio'].iloc[0] = 0


### 如果想要定制特征，处理fut_df \ stock_df即可。

# ADF验证是否平稳

In [4]:
adf_list = []
for idx, df in enumerate(stock_df):
   adf_list.append(adf_test(df[feature]))
adf_df = pd.concat(adf_list, axis=1)   

Test Statistic                  -33.507513
p-value                           0.000000
#Lags Used                        0.000000
Number of Observations Used    1189.000000
Critical Value (1%)              -3.435862
Critical Value (5%)              -2.863974
Critical Value (10%)             -2.568066
dtype: float64
Test Statistic                -1.370201e+01
p-value                        1.284425e-25
#Lags Used                     7.000000e+00
Number of Observations Used    1.186000e+03
Critical Value (1%)           -3.435876e+00
Critical Value (5%)           -2.863980e+00
Critical Value (10%)          -2.568069e+00
dtype: float64
Test Statistic                -1.434570e+01
p-value                        1.041917e-26
#Lags Used                     7.000000e+00
Number of Observations Used    1.176000e+03
Critical Value (1%)           -3.435923e+00
Critical Value (5%)           -2.864001e+00
Critical Value (10%)          -2.568080e+00
dtype: float64
Test Statistic                  -33.48

In [5]:
adf_df

,0,1,2,3,4
Test Statistic,-33.507513,-1.370201e+01,-1.434570e+01,-33.486238,-34.303758
p-value,0.000000,1.284425e-25,1.041917e-26,0.000000,0.000000
#Lags Used,0.000000,7.000000e+00,7.000000e+00,0.000000,0.000000
Number of Observations Used,1189.000000,1.186000e+03,1.176000e+03,1193.000000,1193.000000
Critical Value (1%),-3.435862,-3.435876e+00,-3.435923e+00,-3.435843,-3.435843
Critical Value (5%),-2.863974,-2.863980e+00,-2.864001e+00,-2.863966,-2.863966
Critical Value (10%),-2.568066,-2.568069e+00,-2.568080e+00,-2.568061,-2.568061


对于changeratio而言，是平稳的

# 根据模型找出各个stock的联动显著时间段
* 由于只有VAR，故不再对模型选择进行分支。
* 由于p=0.05基本找不到显著的时间段，改成了0.10
* y是股票的y。第一个p是const的，第二个是期货前一时刻特征的p，第三个是股票前一时刻的特征的p。

## 先算一下相关性
* 这个只算了一个期货和具体股票某个特征的收益。
* 看下面收益率的corr，相关性还挺高。
* 与此同时，把数据的describe也写上了。

In [5]:
corr_list = []
data_describe = []
for df in stock_df:
    data = data_generator(fut_df, df, feature)
    corr_list.append(data.corr())
    data_describe.append(data.describe())
# corr_list
data_describe

[       ChangeRatio_df1  ChangeRatio_df2
 count      1190.000000      1190.000000
 mean          0.000397         0.001785
 std           0.010978         0.025930
 min          -0.062057        -0.100233
 25%          -0.005413        -0.013432
 50%           0.000286         0.000849
 75%           0.006051         0.014644
 max           0.054413         0.100358,
        ChangeRatio_df1  ChangeRatio_df2
 count      1194.000000      1194.000000
 mean          0.000396         0.000705
 std           0.010960         0.023686
 min          -0.062057        -0.100045
 25%          -0.005360        -0.009709
 50%           0.000286         0.000000
 75%           0.006028         0.011032
 max           0.054413         0.100206,
        ChangeRatio_df1  ChangeRatio_df2
 count      1184.000000      1184.000000
 mean          0.000421         0.000723
 std           0.010981         0.023142
 min          -0.062057        -0.099773
 25%          -0.005299        -0.010390
 50%          

#### 主意这个找显著，都是90%置信读就行。因为95%的找不到。

In [6]:
sig_col = []
for idx, df in enumerate(stock_df):
    data = data_generator(fut_df, df, feature)
    sig_col.append(significant(data))
sig_col
# 要画什么图根据这个再改一改。

[[[0, array([0.07669832, 0.03720587, 0.01438783])],
  [1, array([0.07110896, 0.03989815, 0.02220851])],
  [3, array([0.03700736, 0.02427026, 0.03001648])],
  [461, array([0.06187387, 0.01776586, 0.09157615])],
  [462, array([0.07216438, 0.01201078, 0.06453823])],
  [543, array([0.09130337, 0.00218921, 0.0152069 ])],
  [846, array([0.04396597, 0.08938499, 0.0434864 ])],
  [847, array([0.02516317, 0.07404043, 0.03527982])]],
 [[3, array([0.09983438, 0.02048287, 0.02625046])],
  [5, array([0.06254327, 0.0106275 , 0.03478245])],
  [6, array([0.0279161 , 0.01415837, 0.0375582 ])],
  [137, array([0.04668035, 0.09926507, 0.06327435])],
  [138, array([0.07776827, 0.03992777, 0.04845521])],
  [139, array([0.06423892, 0.03505261, 0.05556232])],
  [141, array([0.09273869, 0.0355535 , 0.07064869])],
  [142, array([0.09610814, 0.03836194, 0.07086439])],
  [256, array([0.02002243, 0.0120848 , 0.01287456])],
  [357, array([0.00793185, 0.09517341, 0.05815873])],
  [724, array([0.03831802, 0.06976187, 

上面sig为list。list 0-4表明对应的stock。

比如list[0]中，就存了期股联动显著的时间切片。list中第一个数字是时间切片编号，第二个是三个p。p的解释如上

sig_col 即为显著的index

还有更进一步的算最佳lead lag,不过都是1，所以似乎不重要了,写在下面。可以尝试下冲击

```python
lag_order = model.select_order(15)
print(f"Selected lag order: {lag_order.selected_orders['aic']}")
model_fitted = model.fit(maxlags=lag_order.selected_orders['aic'])
print(model_fitted.summary())

irf = model_fitted.irf(10) # 10期冲击响应
irf.plot(orth=True) # 正交化冲击响应图
plt.show()

```

# 协整检验
* 只用检验上面的data数据（就是一列是期货数据，一列是股票数据）

!! 只看一次，所以只用了一个break

In [7]:
# print('Results of Johansen Cointegration Test:')
# print(f"Test statistic: {result.lr1}")
# print(f"Critical values: {result.cvt}")
# print(f"Eigenstatistics: {result.lr2}")
# print(f"Eigenvalues: {result.eig}")
Test_statistic = []
Critical_values = []
Eigenstatistic = []
Eigenvalues = []
for idx, df in enumerate(stock_df):
    data = data_generator(fut_df, df, feature)
    test_result = johansen_cointegration_test(data)
    Test_statistic.append(test_result.lr1)
    Critical_values.append(test_result.cvt)
    Eigenstatistic.append(test_result.lr2)
    Eigenvalues.append(test_result.eig)

# 可以很方便的改成df

# E-G协整检验

* 对回归残差的检验

* result是股票对期货回归得到的模型的参数。比如t值 p值 r^2等

In [7]:
result_list = []
output_list = []
for df in stock_df:
    data = data_generator(fut_df, df, feature)
    result_df, output = EG(data)
    result_list.append(result_df)
    output_list.append(output)


In [6]:
result_df

,Coefficients,t-values,p-values,r_squared
const,-0.000013,-0.021723,9.826723e-01,0.32489
ChangeRatio_df1,1.257340,23.950747,8.209296e-104,0.32489


# ATE（暂时失败）

In [4]:
import numpy as np
from causalinference import CausalModel

# 生成示例数据
np.random.seed(0)
n = 1000
A = np.random.normal(loc=0, scale=1, size=n)
B = 2 * A + np.random.normal(loc=0, scale=1, size=n)

# 由于示例中没有协变量X，我们可以创建一个全为1的数组作为占位符
# 这样做是为了满足CausalModel的参数要求
X = np.ones((n, 1))

# 创建因果推断模型
# 注意：这里我们直接传递Y（结果变量B）、D（处理变量A）和X（协变量）
causal_model = CausalModel(Y=B, D=A, X=X)

# 进行因果推断分析
causal_model.est_via_ols(adj=1)
print(causal_model.estimates)

# 获取平均因果效应（ATE）
ate = causal_model.estimates['ols']['ate']
print("Average Treatment Effect (ATE):", ate)


ValueError: Too few treated units: N_t < K+1